# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,gympie,-26.1898,152.6650,21.36,84,81,0.90,AU,1701798670
1,1,douglas,41.3000,-96.1670,6.02,64,0,7.20,US,1701798612
2,2,jamestown,42.0970,-79.2353,1.77,98,100,0.45,US,1701798757
3,3,adamstown,-25.0660,-130.1015,22.52,82,100,6.33,PN,1701798812
4,4,ancud,-41.8697,-73.8203,15.40,74,13,3.03,CL,1701798549


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map1 = city_data_df.hvplot.points('Lng', "Lat", geo=True, tiles="OSM"
                              ,frame_width = 700, frame_height = 500, color="City", size="Humidity")

# Display the map
map1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
 

ideal_weather_df = city_data_df[(city_data_df['Max Temp'] >=21) & (city_data_df['Max Temp'] <=30) &
                                (city_data_df['Cloudiness'] == 0) & (city_data_df['Wind Speed'] <= 4.5) ]

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
86,86,agadez,19.7500,10.2500,21.02,21,0,2.43,NE,1701798824
125,125,los angeles,34.0522,-118.2437,26.13,38,0,3.60,US,1701798283
126,126,sur,22.5667,59.5289,24.55,45,0,3.61,OM,1701798829
154,154,al jubayl,27.0112,49.6583,21.75,56,0,3.49,SA,1701798831
162,162,saint-pierre,-21.3393,55.4781,22.82,78,0,2.57,RE,1701798832
194,194,la cruz,23.9167,-106.9000,27.09,38,0,2.33,MX,1701798835
210,210,santa ana,33.7456,-117.8678,26.81,45,0,0.00,US,1701798276
232,232,pisco,-13.7000,-76.2167,23.03,73,0,4.12,PE,1701798839
261,261,zayed city,24.3768,54.5890,27.22,71,0,3.60,AE,1701798843
292,292,coyhaique,-45.5752,-72.0662,23.34,35,0,2.57,CL,1701798849


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
columns_to_copy = ['City', 'Country', 'Lat', 'Lng', 'Humidity']
hotel_df= ideal_weather_df[columns_to_copy].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']= ''

# Display sample data
hotel_df.head()
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
86,agadez,NE,19.7500,10.2500,21,
125,los angeles,US,34.0522,-118.2437,38,
126,sur,OM,22.5667,59.5289,45,
154,al jubayl,SA,27.0112,49.6583,56,
162,saint-pierre,RE,-21.3393,55.4781,78,
194,la cruz,MX,23.9167,-106.9000,38,
210,santa ana,US,33.7456,-117.8678,45,
232,pisco,PE,-13.7000,-76.2167,73,
261,zayed city,AE,24.3768,54.5890,71,
292,coyhaique,CL,-45.5752,-72.0662,35,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
import requests

# Set parameters to search for a hotel
radius = 10000
params = {"apiKey": geoapify_key, "format": "json"}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    long = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{long},{lat},{radius}"
    params["bias"] = f"proximity:{long},{lat}"
    params["categories"] = "accommodation.hotel"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address_response = requests.get(base_url, params=params)
    
     # Convert the API response to JSON format
    name_address_json = name_address_response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address_json["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
agadez - nearest hotel: No hotel found
los angeles - nearest hotel: DoubleTree by Hilton Hotel Los Angeles Downtown
sur - nearest hotel: Sur Hotel
al jubayl - nearest hotel: Sharq
saint-pierre - nearest hotel: Tropic Hotel
la cruz - nearest hotel: Hotel Las Palmas
santa ana - nearest hotel: Holiday Inn Express
pisco - nearest hotel: La Portada
zayed city - nearest hotel: L'Arabia hotel
coyhaique - nearest hotel: No hotel found
san jose del cabo - nearest hotel: Hotel Posada Terranova
aswan - nearest hotel: Yaseen hotel
piti village - nearest hotel: The Cliff Hotel
khawr fakkan - nearest hotel: Raynor Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
86,agadez,NE,19.7500,10.2500,21,No hotel found
125,los angeles,US,34.0522,-118.2437,38,DoubleTree by Hilton Hotel Los Angeles Downtown
126,sur,OM,22.5667,59.5289,45,Sur Hotel
154,al jubayl,SA,27.0112,49.6583,56,Sharq
162,saint-pierre,RE,-21.3393,55.4781,78,Tropic Hotel
194,la cruz,MX,23.9167,-106.9000,38,Hotel Las Palmas
210,santa ana,US,33.7456,-117.8678,45,Holiday Inn Express
232,pisco,PE,-13.7000,-76.2167,73,La Portada
261,zayed city,AE,24.3768,54.5890,71,L'Arabia hotel
292,coyhaique,CL,-45.5752,-72.0662,35,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map2 = hotel_df.hvplot.points('Lng', "Lat", geo=True, tiles="OSM"
                              ,frame_width = 700, frame_height = 500, color="City", hover_cols=['City', 'Country', 'Hotel Name', 'Humidity'])

# Display the map
map2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name,Humidity)